In [1]:
import pandas as pd
import json
import pathlib
import os
import dotenv
import time

In [2]:
# get variables
dotenv.load_dotenv()
working_dir = pathlib.Path.cwd()
LABEL_STUDIO_HOST = "https://label.drgoktugasci.com"
ANNOTATIONS_PATH = working_dir.parent / "data" / "output" / "annotations.json"

# start the heroku server if stopped
os.system("heroku ps:scale web=1 --app er-reports")
time.sleep(15)

# get the annotations from the label studio server
os.system(
    f"""
    curl \
    -X GET {LABEL_STUDIO_HOST}/api/projects/4/export?exportType=JSON \
    -H 'Authorization: Token {os.getenv("LABEL_STUDIO_API_KEY")}' \
    --output {ANNOTATIONS_PATH.as_posix()}
    """
)

 ›   Warning: heroku update available from 7.60.2 to 8.1.9.
Scaling dynos... done, now running web at 1:Basic
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  139k  100  139k    0     0  64065      0  0:00:02  0:00:02 --:--:-- 64201


0

In [3]:
# Read the JSON data from file
with open(ANNOTATIONS_PATH, "r") as file:
    json_data = json.load(file)

# Create a list to store flattened data
flattened_data_list = []

# Flatten the nested data for each entry in the JSON data
for entry in json_data:
    annotations = entry.get("annotations", [])
    data = entry.get("data", [])
    for annotation in annotations:
        result = annotation.get("result", [])
        for res in result:
            value = res.get("value", {})
            flattened_data = {
                # "id": entry["id"],
                "annotation_id": annotation["id"],
                # "completed_by": annotation["completed_by"],
                "type": res["type"],
                # "end": value.get("end", None),
                # "start": value.get("start", None),
                "text": value.get("text", None),
                "labels": value.get("labels", None),
                "choices": value.get("choices", None),
                # "origin": res.get("origin", None),
                # "to_name": res.get("to_name", None),
                # "from_name": res.get("from_name", None),
                "full_text": data.get("text", None),
                "study_no": data.get("study_no", None),
                "patient_no": data.get("patient_no", None),
                "report_date": data.get("report_date", None),
                "report_count": data.get("report_count", None),
            }
            flattened_data_list.append(flattened_data)

# Convert to a Pandas DataFrame
df = pd.DataFrame(flattened_data_list)

In [4]:
# separate labels and choices
df_labels = df.loc[df.type == "labels"]
df_choices = df.loc[df.type == "choices"]

# merge labels & choices & select only the relevant columns
df_labels_choices = (
    (
        df_labels.groupby("annotation_id")
        .agg({"text": " ".join})
        .reset_index()
        .merge(df_choices, on="annotation_id", how="left")
    )[
        [
            "annotation_id",
            # "study_no",
            "patient_no",
            "report_date",
            # "report_count",
            "full_text",
            "text_x",
            "choices",
        ]
    ]
    .rename(columns={"text_x": "relevant_text", "choices": "classifications"})
)

# filter excluded ones
df_final = df_labels_choices.loc[~df_labels_choices["classifications"].str.contains("Exclude", na=False)]

In [19]:
df_final["relevant_text"][0]

'CPA ve prepontin sisternler geniş izlenmektedir. Serebellar folyalar belirgindir. 3. ventrikül ve lateral ventriküller geniş olarak izlenmektedir. Serebral sulkus ve sisternalar genişlemiştir. Periventriküler alanlarda lökoriazis ile uyumlu görünüm mevcuttur.'

In [17]:
# add word count to the data frame
df_final["word_count"] = (
    df_final["relevant_text"]
    .apply(lambda x: len(str(x).split()))
)

# word count ascending
df_final.sort_values(["word_count"], ascending=False)

,annotation_id,patient_no,report_date,full_text,relevant_text,classifications,word_count
18,36,2004355719,2021-06-11,Kontrastsız Beyin BT Kafa kaidesi ve kalvaria...,Kontrastsız Beyin BT Kafa kaidesi ve kalvarial...,[Normal],78
4,19,2008696775,2020-10-13,KONTRASTSIZ BEYİN BT İnfratentorial kesitler...,Sol serebellar hemisferde sekel ??? ensefaloma...,[Non Emergency [No Doctor]],60
12,28,2006088545,2021-06-07,Kontrastsız Beyin BT tetkiki Kafa kaidesi v...,Periventriküler derin beyaz cevherde kronik i...,[Non Emergency [No Doctor]],57
47,74,2009366291,2021-06-23,Beyin BT ve orbita BT tetkiklerinde Posterio...,Sağda orbita tavanınd frontal sinüs tabanında ...,[Emergency],54
37,61,2009021658,2021-06-22,Beyin BT tetkiki Kafa kaidesi normal sınırl...,Sağ orbita üst duvarında fraktüre ait görünüm ...,[Emergency],53
15,31,2008704136,2021-06-10,BEYİN BT Posterior fossada beyin sapı ar...,BEYİN BT Posterior fossada beyin sapı artefatk...,[Normal],53
31,53,2008247651,2021-06-16,Kafa tabanı ve verteks arasından elde olunan k...,Kafa tabanı ve verteks arasından elde olunan k...,[Normal],52
22,44,2009201466,2021-06-14,Kafa tabanı ve verteks arasından elde olunan k...,Kafa tabanı ve verteks arasından elde olunan k...,[Normal],52
28,50,2006053732,2021-06-16,Kafa tabanı ve verteks arasından elde olunan k...,Kafa tabanı ve verteks arasından elde olunan k...,[Normal],52
29,51,2008211024,2021-06-16,Kafa tabanı ve verteks arasından elde olunan k...,Kafa tabanı ve verteks arasından elde olunan k...,[Normal],52


,annotation_id,patient_no,report_date,full_text,relevant_text,classifications,word_count
0,15,2008251410,2010-05-24,BEYIN BT TETKİKİ prt no 9644 Kafa tabanı...,CPA ve prepontin sisternler geniş izlenmektedi...,[Non Emergency [No Doctor]],29
1,16,2007066964,2020-10-06,Kafa tabanı ve verteks arasından elde olunan k...,CPA ve prepontin sisternler geniş izlenmekted...,[Non Emergency [No Doctor]],22
2,17,2008043690,2020-10-06,Kafa tabanı ve verteks arasından elde olunan k...,Sağ sylvian fissur komşuluğunda kortikal düzey...,[Non Emergency [No Doctor]],10
3,18,2008037641,2020-10-13,KONTRASTSIZ BEYİN BT İnfratentorial kesitler...,Sağda bazal ganglionik düzeyde genişlemiş peri...,[Non Emergency [No Doctor]],13
4,19,2008696775,2020-10-13,KONTRASTSIZ BEYİN BT İnfratentorial kesitler...,Sol serebellar hemisferde sekel ??? ensefaloma...,[Non Emergency [No Doctor]],60
5,20,2009516271,2020-10-13,KONTRASTSIZ BEYİN BT İnfratentorial kesitler...,Supratentorial kesitlerin değerlendirilmesinde...,[Non Emergency [No Doctor]],20
6,21,2004408373,2021-02-01,"BT, BEYIN, KONTRASTSIZ Kafa tabanı ve verteks...",Bilateral etmoid selüllerde havlanma kaybına n...,[Non Emergency [No Doctor]],25
7,22,2009461379,2021-02-06,Posterior fossada beyin sapı artefatklara bağl...,Kemik pencere yapılan incelemede sağ frontalde...,[Non Emergency [No Doctor]],24
8,24,2005066801,2021-06-01,Beyin BT tetkiki Kafa kaidesi ve kalvarial ...,Verteks düzeyinde sağ orta hatta cilt- cilt al...,[Non Emergency [No Doctor]],13
9,25,2007091932,2021-06-01,BEYİN BT Kafa tabanı ve verteks arasınd...,BEYİN BT Kafa tabanı ve verteks arasından elde...,[Normal],51
